<a href="https://colab.research.google.com/github/sc22lg/COMP3611_Machine_Learning/blob/main/COMP3611_202526_Set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursework COMP3611 (Winter 2025)
**A pdf brief with submission instructions can be found in the same folder as this notebook. Please follow these instructions carefully**

This coursework notebook contains 2 questions. The total of all subquestions is worth 30 marks.

**All your answers need text cells!** *Comments in code do not count as answers*. Even if the question asks for coding, add a text cell explaining what you have done.

## Question 1 - Principal Component Analysis

**Q1 a (4 marks)**

You are given a data array called "shape_array.npy" that comprises 7 samples organised as columns in the array, where each column corresponds to one sample. The data format in each column is: [x_1, y_1, z_1, x_2, y_2, z_2, ………, x_N, y_N, z_N], where (x_i, y_i, z_i) corresponds to the i-th 3D point of a blood vessel. By plotting all 3D points in one column, you can obtain the shape of a blood vessel of that sample.

Plot seven figures to show the 3D blood vessel shape for each sample separately. Also plot two arbitrary shapes on top of each other to get a feeling of how similar or dissimilar the shapes are.

In [44]:
import plotly.express as px
import numpy as np
import pandas as pd

shape_array = np.load('shape_array.npy')
print(shape_array.shape)

def get_display_datapoint(data):

  x = data[0::3]
  y = data[1::3]
  z = data[2::3]

  df = pd.DataFrame({"x": x, "y": y, "z": z})
  return df


for i in range(7):
  df = get_display_datapoint(shape_array.T[i])
  fig = px.line_3d(df, x = "x", y = "y", z = "z")
  fig.show()

(1845, 7)


In the code above we extract every 3rd element from each collumn to find X, Y and Z co-ordinates separately.
Then for each of the columns we construct a dataframe of co-ordinates, we then plot this in 3D.

In [45]:
df1_vessel = get_display_datapoint(shape_array.T[0])
df2_vessel = get_display_datapoint(shape_array.T[1])

df = pd.concat([df1_vessel, df2_vessel])
fig = px.line_3d(df, x = "x", y = "y", z = "z")
fig.show()

In the code above we construct a dataframe for two arbitrary shapes and give them distinct line names. We then concatenate these into one dataframe which can be plotted in 3D.

**Q1 b (10 marks)**

Next, perform eigendecomposition of the covariance matrix estimated from the given data array. Finally, project original data onto lower-dimensional space and reconstruct data.

Proceed as follows:

1. Subtract the mean from the data, so that it is centered around the origin.

2. Estimate the covariance matrix from the centred data.

3. Calculate eigenvectors and eigenvalues using numpy functions

4. Project centered data (1845 dimension) into a lower-dimension space (You need to choose a reasonable dimension).

5. Reconstruct the blood vessel shape from the lower dimension data in step 4.

As a sanity check plot a blood vessel shape reconstructed from the eigenvectors on top of the original blood vessel shape. Explain how much data reduction you have achieved. Comment on your results.

In [46]:
print(shape_array.shape)
data_points = shape_array.T
print(data_points.shape)

#center datapoints
means = np.mean(data_points, axis = 0)
print(means.shape)
data_points_centered = data_points - means
print(data_points_centered.shape)

df = get_display_datapoint(data_points_centered[2])
fig = px.line_3d(df, x = "x", y = "y", z = "z")
fig.show()

(1845, 7)
(7, 1845)
(1845,)
(7, 1845)


In [47]:
#find covariance matrix
covariance_estimate = np.cov(data_points_centered.T)
print(covariance_estimate.shape)
#find eigen values and vectors
eig_vals, eig_vecs = np.linalg.eig(covariance_estimate)
print(eig_vals.shape)

(1845, 1845)
(1845,)


In [48]:
#express dataset in eigenbasis
eigenbasis = eig_vecs.T.dot(data_points_centered.T).real
print(eigenbasis.shape)

(1845, 7)


###Get Highest eigenvalues:

In [49]:
n_highest = 9
ascending_value_indicies = np.argsort(eig_vals)
top_value_indicies = ascending_value_indicies[-n_highest:]
print(top_value_indicies)

[23 10  6  5  4  3  2  1  0]


###Perform Reduction by setting to 0 at low eigval positions

In [50]:
zeroes = np.zeros(eigenbasis.shape[1])
reduced_data = np.zeros_like(eigenbasis)

for i in top_value_indicies:
  print(eig_vals[i].real)
  reduced_data[i] = eigenbasis[i]

7.874389799252622e-20
1.0033416554652035e-19
1.507937626674753e-19
1.1333163509497034e-05
3.120369718041285e-05
0.00011443539102089196
0.00014392445133842112
0.00038329801540105284
0.0011398973067329617


###Project reduced dataset back to original co-ordinates

In [51]:
#get inverse of eigvecs
einvec_inv = np.linalg.inv(eig_vecs)

#apply inverse of eigvecs to 'unrotate'
reduced_shapes = einvec_inv.T.dot(reduced_data).real
print(reduced_shapes.shape)

(1845, 7)


In [52]:
#re-add the means
reduced_shapes_un_centered = reduced_shapes + means[:, np.newaxis]
print(reduced_shapes_un_centered.shape)

(1845, 7)


In [53]:
for i in range(7):
  df = get_display_datapoint(reduced_shapes_un_centered.T[i])
  fig = px.line_3d(df, x = "x", y = "y", z = "z")
  fig.show()

In [54]:
#plot datapoints over eachother
reduced_vessel = get_display_datapoint(reduced_shapes_un_centered.T[1])
df = pd.concat([df2_vessel, reduced_vessel])
fig = px.line_3d(df, x = "x", y = "y", z = "z")
fig.show()

##Results:
By plotting the eigenvalues we can see that only the top 6-10 eigenvalues have any significance as they fall into the x10^-20 range around the 9th least significant out of 1845. After deciding how many eigenvalues we want to consider (in this case 9), and transforming our centered dataset into the eigenbasis, we perform a reduction by setting the least signigicant dimensions of the dataset to all 0s.

We now have 9 dimensions representing our dataset which was 1845 dimensions.
The resulting data reduction performed here is a 99.5% reduction.

We then project (rotate) our dataset back into our original basis using the inverse of the eigenvectors (as the eigenvector matrix is symetric, its transpose acts as an inverse rotation). We then add back in our mean vectors to un-center the data. Displaying the reconstruction with the original shows that the difference between them is negligible despite the 99.5% reduction.

**Q1 c (4 marks)**

Research PCA analysis using the *scikit-learn* library. Perform PCA analysis and show the reconstructed data of any blood vessel shape on top of the
original blood vessel shape. There are variables in the PCA object that correspond to the eigenvalues used for choosing projection eigenvectors. Compare the eigenvalues  and eigenvectors you have computed in the previous question with the eigenvalues  and the eigenvectors computed by the *scikit-learn* library. Compare the reconstructed coordinates from both methods. Comment on your results.

In [55]:
from sklearn.decomposition import PCA

pca = PCA(n_components=7)
data_reduced = pca.fit_transform(data_points)

In [56]:
print(data_reduced)
print(pca.mean_.shape)
print(pca.explained_variance_)

[[-8.65867268e-03  8.03142879e-03 -5.70059195e-03  5.20327501e-03
   1.15087032e-02  1.51202024e-03  1.90123455e-08]
 [ 4.78102192e-02  2.49408633e-02  2.03562179e-03  6.21948903e-03
  -4.22784965e-03  2.55284668e-03  1.90122815e-08]
 [ 3.52272429e-02 -1.52638685e-02 -8.10757000e-03 -1.83543283e-02
   8.45023373e-04 -5.09417790e-04  1.90122869e-08]
 [ 3.48244654e-03 -3.39014009e-02  1.22609427e-02  9.48666129e-03
  -8.24641960e-04  1.78459438e-03  1.90122655e-08]
 [-4.86487262e-02  1.21018253e-02  1.02909468e-02 -1.17555019e-02
  -2.37111677e-03  2.52132677e-03  1.90122993e-08]
 [ 9.27973233e-05  8.21503717e-03  9.55761783e-03  2.00574589e-03
   5.12171537e-04 -6.97155949e-03  1.90122442e-08]
 [-2.93052923e-02 -4.12394898e-03 -2.03369651e-02  7.19468482e-03
  -5.44233667e-03 -8.89833667e-04  1.90122478e-08]]
(1845,)
[1.13989727e-03 3.83298262e-04 1.43924553e-04 1.14435425e-04
 3.12037737e-05 1.13331744e-05 4.21711367e-16]


In [57]:
# To reconstruct the data from the reduced dimension (and add mean): pca.inverse_transform().
data_reconstructed_un_centered = pca.inverse_transform(data_reduced)
print(f"Shape of data_reconstructed_un_centered: {data_reconstructed_un_centered.shape}")

Shape of data_reconstructed_un_centered: (7, 1845)


In [58]:
scikit_reduced_vessel = get_display_datapoint(data_reconstructed_un_centered[1])
df = pd.concat([scikit_reduced_vessel, reduced_vessel])
fig = px.line_3d(df, x = "x", y = "y", z = "z")
fig.show()

###Comparison
Eigenvalue used to choose eigenvectors:

*   Scikitlearn: [1.13989727e-03, 3.83298262e-04, 1.43924553e-04, 1.14435425e-04,
 3.12037737e-05, 1.13331744e-05, 4.21711367e-16]
*   Part b: [0.00113989730, 0.000383298015, 0.000143924451, 0.000114435391, 3.12036971e-05, 1.13331635e-05, 1.50793762e-19, 1.00334165e-19, 7.87438979e-20]

This shows that the eigenvalues calculated to be used to choose projection eigenvectors are very similar until the 7th value onwards, with also some slight differences in the calculated values after x10^-9 e.g. the digit after the first '7' in each of the first values is different. This could be due to slightly different methods between the implementation in part b and Scikitlearn.

Also in the implementation for part B, I chose to include two addidional dimensions (9 instead of scikitlearn's 7) in order to get the tail end of the significant variences.

###Reconstruction
As well as this, as shown in the reconstructions above, there is no visible difference between the two reconstructed bloodvessels from scikitlearn and part b, validating my previous method. While there is no visible difference in the graph, zooming in closely and hovering around a specific point (or printing off the points individually), it can be seen that there is a very small difference between the two, which is expected given the differences in eigenvalues used to determine the reconstruction, as described above.


# Question 2 : Predict Cancer Mortality Rates in US Counties

The provided dataset comprises data collected from multiple counties in the US. The regression task for this assessment is to predict cancer mortality rates in "unseen" US counties, given some training data. The training data ('Training_data.csv') comprises various features/predictors related to socio-economic characteristics, amongst other types of information for specific counties in the country. The corresponding target variables for the training set are provided in a separate CSV file ('Training_data_targets.csv'). Use the notebooks provided for lab sessions throughout this module to provide solutions to the exercises listed below. Throughout all exercises text describing your code and answering any questions included in the exercise descriptions should be included as part of your submitted solution.


The list of predictors/features available in this data set are described below:

**Data Dictionary**

avgAnnCount: Mean number of reported cases of cancer diagnosed annually

avgDeathsPerYear: Mean number of reported mortalities due to cancer

incidenceRate: Mean per capita (100,000) cancer diagoses

medianIncome: Median income per county

popEst2015: Population of county

povertyPercent: Percent of populace in poverty

MedianAge: Median age of county residents

MedianAgeMale: Median age of male county residents

MedianAgeFemale: Median age of female county residents

AvgHouseholdSize: Mean household size of county

PercentMarried: Percent of county residents who are married

PctNoHS18_24: Percent of county residents ages 18-24 highest education attained: less than high school

PctHS18_24: Percent of county residents ages 18-24 highest education attained: high school diploma

PctSomeCol18_24: Percent of county residents ages 18-24 highest education attained: some college

PctBachDeg18_24: Percent of county residents ages 18-24 highest education attained: bachelor's degree

PctHS25_Over: Percent of county residents ages 25 and over highest education attained: high school diploma

PctBachDeg25_Over: Percent of county residents ages 25 and over highest education attained: bachelor's degree

PctEmployed16_Over: Percent of county residents ages 16 and over employed

PctUnemployed16_Over: Percent of county residents ages 16 and over unemployed

PctPrivateCoverage: Percent of county residents with private health coverage

PctPrivateCoverageAlone: Percent of county residents with private health coverage alone (no public assistance)

PctEmpPrivCoverage: Percent of county residents with employee-provided private health coverage

PctPublicCoverage: Percent of county residents with government-provided health coverage

PctPubliceCoverageAlone: Percent of county residents with government-provided health coverage alone

PctWhite: Percent of county residents who identify as White

PctBlack: Percent of county residents who identify as Black

PctAsian: Percent of county residents who identify as Asian

PctOtherRace: Percent of county residents who identify in a category which is not White, Black, or Asian

PctMarriedHouseholds: Percent of married households

BirthRate: Number of live births relative to number of women in county

In [59]:
import os
import pandas as pd

## Define paths to the training data and targets files
root_dir = './'
training_data_path = root_dir + 'Training_data.csv'
training_targets_path = root_dir + 'Training_data_targets.csv'

**Q2 a**

Read in the training data and targets files. The training data comprises features/predictors while the targets file comprises the targets (i.e. cancer mortality rates in US counties) you need to train models to predict. Plot histograms of all features to visualise their distributions and identify outliers. Do you notice any unusual values for any of the features? If so comment on these in the text accompanying your code. Compute correlations of all features with the target variable (across the data set) and sort them according the strength of correlations. Which are the top five features with strongest correlations to the targets? Plot these correlations using the scatter matrix plotting function available in pandas and comment on at least two sets of features that show visible correlations to each other.

**(4 marks)**

###Unusual Values:

avgAnnCount - one value of 25k, outlier

avg deaths per year - one value of 10k, outlier

incidence rate - one value 1200, outlier

StudyPerCap- Was told to ignore this feature as not described in the data dictionary - outliers around 9k

Median Age has values in the 350 - 600 year range, obviously incorrect as no-one lives tha long, oldest person ever was https://en.wikipedia.org/wiki/Jeanne_Calment who lived to be 122.

AvgHouseholdSize - 48 counties with 0

Birthrates up to 21 is unrealistic. As well is the median being around 5 as historically the US average is less than 2.

####Outlier Observation
Many of these outliers could be caused by one county having a disproportionaly large population of people for example dense urban counties like New York County (Manhattan). Meaning these outliers are not necessarily mistakes.

In [60]:
training_data = pd.read_csv('Training_data.csv')
training_data_targets = pd.read_csv('Training_data_targets.csv')

for column in training_data.columns:
    fig = px.histogram(training_data, x=column, nbins=50, title=f"Histogram of {column}")
    fig.show()


In [61]:
#finding correlation
correlation_matrix = training_data.corrwith(training_data_targets['TARGET_deathRate'])

top5 = correlation_matrix.reindex(
    correlation_matrix.abs().sort_values(ascending=False).head(5).index
)

print(top5)


PctBachDeg25_Over        -0.491411
incidenceRate             0.443983
PctPublicCoverageAlone    0.439734
medIncome                -0.416607
povertyPercent            0.413260
dtype: float64


In [62]:
# Combine top 5 features and the target into a single DataFrame for plotting
features_to_plot = top5.index.tolist()
plot_df = training_data[features_to_plot].copy()
plot_df['TARGET_deathRate'] = training_data_targets['TARGET_deathRate']

# Plot individual scatter plots for each of the top 5 features against TARGET_deathRate
for feature in features_to_plot:
    fig = px.scatter(plot_df, x=feature, y='TARGET_deathRate', title=f"Scatter Plot of {feature} vs. TARGET_deathRate")
    fig.show()

##Correlations:
The top 5 correlated features are:

*   PctBachDeg25_Over        -0.491411
*   incidenceRate             0.443983
*   PctPublicCoverageAlone    0.439734
*   medIncome                -0.416607
*   povertyPercent            0.413260


The values above are the 5 most strongly correlated values (positive or negatigve)

incidence rate and deathRate have a clear visual correlation, especially if you ignore the outlier datapoints, the scatter plot shows a clear up and to the right trend.

Percentage Bachelors Degree 25 Over also has visible negative correlation with death rate, falling off quickly around 5-10% then flattening out.

**Q2 b**

Create an ML pipeline using scikit-learn (as demonstrated in the lab notebooks) to pre-process the training data.

**(3 marks)**

In [63]:
import sklearn
#from sklearn.model_selection import train_test_split

Test_size = 0.2
seed = 67 #21

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(training_data, training_data_targets, test_size = Test_size, random_state = seed)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

X_train:  (1950, 31)
X_test:  (488, 31)
y_train:  (1950, 1)
y_test:  (488, 1)


Next we must Impute our data

In [65]:
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy="median")
imputer.fit(X_train)

X_train_tr = imputer.transform(X_train)
print(X_train_tr)

[[1.55000000e+02 4.60000000e+01 4.04900000e+02 ... 6.49637085e-01
  6.28430448e+01 7.08007812e+00]
 [1.23000000e+02 5.30000000e+01 4.84200000e+02 ... 6.58668550e-02
  5.27904398e+01 6.33697281e+00]
 [2.40000000e+01 1.00000000e+01 4.29400000e+02 ... 8.00000000e-01
  5.34946237e+01 6.38629283e+00]
 ...
 [1.10700000e+03 4.25000000e+02 4.30000000e+02 ... 7.39523079e+00
  4.33262042e+01 5.34002142e+00]
 [1.12100000e+03 4.51000000e+02 4.82300000e+02 ... 4.06850105e+00
  4.74812419e+01 4.97081868e+00]
 [4.40000000e+01 2.20000000e+01 3.31300000e+02 ... 8.98705255e-01
  5.54483541e+01 3.86165212e+00]]


**Q2 c**

Perform linear regression on the target data. Make a taining/test splot of 80/20. Train a linear classifier on the training set. Evaluate the root mean squared error on both the test and training set. For both the training and the test data plot the predicted value vs the actual target value.  Discuss on the basis of these results whether you believe that the model overfits, underfits or fits correctly.

**(5 marks)**



In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#fit training data with linear regressor
lin_reg = LinearRegression()
lin_reg.fit(X_train_tr, y_train)

#run prediction on training data
train_predictions = lin_reg.predict(X_train_tr)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions)) # calculate error
print(train_rmse)

19.0359212835574


In [67]:
#use fitted linear regressor to predict on test data
test_predictions = lin_reg.predict(imputer.transform(X_test))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions)) # calculate error
print(test_rmse)

18.715717243228603


In [68]:
import matplotlib.pyplot as plt

# Create a DataFrame for plotting
train_plot_df = pd.DataFrame({
    'Actual': y_train['TARGET_deathRate'].values,
    'Predicted': train_predictions.flatten()
})

# Plotting predictions vs target values
fig = px.scatter(train_plot_df, x='Actual', y='Predicted', title='Predicted vs. Actual Target Values (Training Set)')
fig.update_layout(xaxis_title='Actual TARGET_deathRate', yaxis_title='Predicted TARGET_deathRate')
fig.show()

# Create a DataFrame for plotting
test_plot_df = pd.DataFrame({
    'Actual': y_test['TARGET_deathRate'].values,
    'Predicted': test_predictions.flatten()
})

# Plotting predictions vs target values
fig = px.scatter(test_plot_df, x='Actual', y='Predicted', title='Predicted vs. Actual Target Values (Test Set)')
fig.update_layout(xaxis_title='Actual TARGET_deathRate', yaxis_title='Predicted TARGET_deathRate')
fig.show()

In [70]:
test_prediction_correlation = np.corrcoef(test_predictions.flatten(), y_test['TARGET_deathRate'].values)[0, 1]
print(test_prediction_correlation)

0.6909665697028778


###Results:
RMSE: Random Seed 21


*   Train: 18.611262145500593
*   Test: 20.36794600075375

The RMSE for both is similar, with the test error being slightly higher. This is an expected result as the linear regressor has been optimised to fit the training data. However I was also able to observe Test/Train splits that resulted in a lower RMSE for Test than Train.

RMSE: Random Seed 67


*   Train: 19.0359212835574
*   Test: 18.715717243228603

This shows that it is possible for the model to fit 'better' to a test set it hasn't seen before than its training set. This could be due to outliers in the training set that add high error which the smaller test set might not have.

####Plots:
We can see in the plots above that visually the predictions are trending in the correct direction, with predicted rates being higher when the actual rates are also higher (positive linear correlation of 0.691).
This and the RMSE values show that the linear regression model performs well on both training and test sets. This is an indication that a good fit has been found for the data, as if we were over fitting we would see good performance on the training set and poor performance on the test set, and if we were underfitting we would see poor performance on both test and train.